In [2]:
# download and install PLINK2
# see https://www.cog-genomics.org/plink/2.0/
# WARNING: the download link may have to be updated - I couldn't find a stable link
cd /opt/notebooks
wget https://s3.amazonaws.com/plink2-assets/alpha3/plink2_linux_avx2_20220814.zip
unzip -o plink2_linux_avx2_20220814.zip
./plink2 --version

--2022-09-14 12:53:12--  https://s3.amazonaws.com/plink2-assets/alpha3/plink2_linux_avx2_20220814.zip
Resolving s3.amazonaws.com (s3.amazonaws.com)... 52.216.26.22
Connecting to s3.amazonaws.com (s3.amazonaws.com)|52.216.26.22|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 9134949 (8.7M) [application/zip]
Saving to: ‘plink2_linux_avx2_20220814.zip.1’

plink2_linux_avx2_2 100%[===================>]   8.71M  14.3MB/s    in 0.6s    

2022-09-14 12:53:13 (14.3 MB/s) - ‘plink2_linux_avx2_20220814.zip.1’ saved [9134949/9134949]

Archive:  plink2_linux_avx2_20220814.zip
  inflating: plink2                  
PLINK v2.00a3.6LM AVX2 Intel (14 Aug 2022)


In [ ]:
# download and install bgenix
# see https://enkre.net/cgi-bin/code/bgen/doc/trunk/doc/wiki/bgenix.md
cd /opt/notebooks
wget http://code.enkre.net/bgen/tarball/release/bgen.tgz
tar xvfz bgen.tgz > /dev/null
cd bgen.tgz/
./waf configure 
./waf 
./build/test/unit/test_bgen
./build/apps/bgenix -g example/example.16bits.bgen –list
cd /opt/notebooks

In [ ]:
# create shortcuts to genotype directories (i.e. to avoid problems with Jupyter using blanks in file names)

DIR=/mnt/project/Bulk/Imputation/UKB*imputation*from*genotype/
ln -sf $DIR /opt/notebooks/impu

DIR=/mnt/project/Bulk/Genotype*Results/Genotype*calls/
ln -sf $DIR /opt/notebooks/geno


In [3]:
# get the phenotype and covariate files

dx download NMRpheno_Linoleic.Acid_Polyunsaturated.Fatty.Acids.txt
dx download NMRpheno_Linoleic.Acid_Polyunsaturated.Fatty.Acids.cova.txt

# make a link to the default name  
ln -sf NMRpheno_Linoleic.Acid_Polyunsaturated.Fatty.Acids.txt pheno.txt
ln -sf NMRpheno_Linoleic.Acid_Polyunsaturated.Fatty.Acids.cova.txt cova.txt


[===========================================================>] Completed 12,247,394 of 12,247,394 bytes (100%) /opt/notebooks/NMRpheno_Linoleic.Acid_Polyunsaturated.Fatty.Acids.txtt
[===========================================================>] Completed 11,449,492 of 11,449,492 bytes (100%) /opt/notebooks/NMRpheno_Linoleic.Acid_Polyunsaturated.Fatty.Acids.cova.txtt


In [4]:
# look at the file content
head -5 pheno.txt
head -5 cova.txt

FID	IID	nmet1	nmet2	nratio	dmet1	dmet2	dratio
1000068	1000068	-1.06446240533014	-1.57563779976244	0.497911448853712	1.44180215462887	1.93236359143486	-0.490561436805989
1000214	1000214	-1.27363638680437	-1.15212433593209	-0.947445908181468	1.37573453858316	2.03488477906259	-0.659150240479434
1000839	1000839	0.32217083828932	0.824580409920588	-0.830001921599838	1.83999988567087	2.48318375453629	-0.643183868865423
1001128	1001128	0.436464854878345	0.152764270605868	0.776073961495932	1.8716071217376	2.33522597635918	-0.463618854621581
FID	IID	sex	age	age2	cholmed	genoPC1	genoPC2	genoPC3	genoPC4	genoPC5	genoPC6	genoPC7	genoPC8	genoPC9	genoPC10
1000068	1000068	2	59	3481	0	-13.3259	2.9054	-2.37355	0.38822	-1.43668	0.547364	2.46829	0.229228	-0.0457369	-1.27121
1000214	1000214	2	48	2304	0	-11.5671	2.03669	-2.13034	4.28344	-7.1727	0.415289	-1.14447	-1.97925	0.895128	3.97226
1000839	1000839	2	61	3721	0	-10.8416	2.00904	-0.107682	-1.1379	-2.46351	0.595459	1.99684	-0.830125	-1.97182	0.338768
10011

In [5]:
# for testing extract a region from the BGEN file
/opt/notebooks/bgen.tgz/build/apps/bgenix -g impu/ukb22828_c11_b0_v3.bgen -incl-range 11:60000000-62000000 > test.bgen



Welcome to bgenix
(version: 1.1.7, revision )

(C) 2009-2017 University of Oxford

Building query                                              :  (65410/?,2.0s,33322.6/s)
Processing 65410 variants                                   : [******************************] (65410/65410,14.6s,4476.9/s)
bgenix: wrote data for 65410 variants to stdout.

Thank you for using bgenix.


In [6]:
# testing plink

./plink2 --bgen test.bgen ref-first --sample impu/ukb22828_c11_b0_v3.sample \
         --pheno pheno.txt --pheno-name nratio \
         --covar cova.txt --covar-name age,sex,cholmed,genoPC1,genoPC2,genoPC3,genoPC4,genoPC5,genoPC6,genoPC7,genoPC8,genoPC9,genoPC10 \
         --out nratio \
         --mac 100 --maf 0.01 --hwe 1e-15 --mind 0.1 --geno 0.1 \
         --linear hide-covar --adjust 

# show the first lines of the association file
head -5 nratio.nratio.glm.linear

# show the top hits
head -1 nratio.nratio.glm.linear.adjusted


PLINK v2.00a3.6LM AVX2 Intel (14 Aug 2022)     www.cog-genomics.org/plink/2.0/
(C) 2005-2022 Shaun Purcell, Christopher Chang   GNU General Public License v3
Logging to nratio.log.
Options in effect:
  --adjust
  --bgen test.bgen ref-first
  --covar cova.txt
  --covar-name age,sex,cholmed,genoPC1,genoPC2,genoPC3,genoPC4,genoPC5,genoPC6,genoPC7,genoPC8,genoPC9,genoPC10
  --geno 0.1
  --glm hide-covar
  --hwe 1e-15
  --mac 100
  --maf 0.01
  --mind 0.1
  --out nratio
  --pheno pheno.txt
  --pheno-name nratio
  --sample impu/ukb22828_c11_b0_v3.sample

Start time: Wed Sep 14 12:57:19 2022
15544 MiB RAM detected; reserving 7772 MiB for main workspace.
Using up to 8 compute threads.
--bgen: 65410 variants detected, format v1.2.
487409 samples imported from .sample file to nratio-temporary.psam .
--bgen: nratio-temporary.pgen + nratio-temporary.pvar written.
487409 samples (264251 females, 222957 males, 201 ambiguous; 487409 founders)
loaded from nratio-temporary.psam.
65410 variants loaded f

In [5]:
# run associations by chromosome
CFROM=11
CTO=11

CFROM=1
CTO=22

export CHR=$CFROM
while [ $CHR -le $CTO ] ; do

  echo "working on CHR $CHR"
  
  # run PLINK
  
  # using imputed data (too slow, need to reduce samples to NMR set first, and still ...)
  # ./plink2 --bgen impu/ukb22828_c${CHR}_b0_v3.bgen ref-first --sample impu/ukb22828_c${CHR}_b0_v3.sample \
  
  # use genotyped data
  ./plink2 --bfile geno/ukb22418_c${CHR}_b0_v2 \
         --pheno pheno.txt --pheno-name nratio,nmet1,nmet2 \
         --covar cova.txt --covar-name age,sex,cholmed,genoPC1,genoPC2,genoPC3,genoPC4,genoPC5,genoPC6,genoPC7,genoPC8,genoPC9,genoPC10 \
         --out nratio.${CHR} \
         --mac 100 --maf 0.01 --hwe 1e-15 --mind 0.1 --geno 0.1 \
         --linear hide-covar --adjust 
         
  head -2 nratio.${CHR}.nratio.glm.linear.adjusted   
  head -2 nratio.${CHR}.nmet1.glm.linear.adjusted   
  head -2 nratio.${CHR}.nmet2.glm.linear.adjusted   


  ((CHR=CHR+1))
  
done

working on CHR 1
PLINK v2.00a3.6LM AVX2 Intel (14 Aug 2022)     www.cog-genomics.org/plink/2.0/
(C) 2005-2022 Shaun Purcell, Christopher Chang   GNU General Public License v3
Logging to nratio.1.log.
Options in effect:
  --adjust
  --bfile geno/ukb22418_c1_b0_v2
  --covar cova.txt
  --covar-name age,sex,cholmed,genoPC1,genoPC2,genoPC3,genoPC4,genoPC5,genoPC6,genoPC7,genoPC8,genoPC9,genoPC10
  --geno 0.1
  --glm hide-covar
  --hwe 1e-15
  --mac 100
  --maf 0.01
  --mind 0.1
  --out nratio.1
  --pheno pheno.txt
  --pheno-name nratio,nmet1,nmet2

Start time: Wed Aug 17 16:44:58 2022
31310 MiB RAM detected; reserving 15655 MiB for main workspace.
Using up to 16 threads (change this with --threads).
488377 samples (264746 females, 223430 males, 201 ambiguous; 488377 founders)
loaded from geno/ukb22418_c1_b0_v2.fam.
63487 variants loaded from geno/ukb22418_c1_b0_v2.bim.
3 quantitative phenotypes loaded.
Calculating sample missingness rates... 51done.
2 samples removed due to missing genotype

In [59]:
# compute p-gain

cat > compute_pgain.sh <<'EOF'

set -
fic=$1
echo "compute p-gain for" $fic
fmet1=`basename $fic .nratio.glm.linear`.nmet1.glm.linear
fmet2=`basename $fic .nratio.glm.linear`.nmet2.glm.linear

echo "checking if files are present"
[ -f $fic ] || exit 1
[ -f $fmet1 ] || exit 1
[ -f $fmet2 ] || exit 1

echo "checking file sizes"
[ `wc -l $fic | sed 's/ .*//'` -eq `wc -l $fmet1 | sed 's/ .*//'` ] || exit 1
[ `wc -l $fic | sed 's/ .*//'` -eq `wc -l $fmet2 | sed 's/ .*//'` ] || exit 1


echo "getting pvalues"
cut -f12 $fic > prat.$$
cut -f12 $fmet1 > pmet1.$$
cut -f12 $fmet2 > pmet2.$$
paste pmet1.$$ pmet2.$$ prat.$$ > p3.$$

echo "computing pgain"
cat p3.$$ | awk '{
if ( $1 == "P" ) { 
  print "PGAIN" 
} else {
  x1 = $1/$3
  x2 = $2/$3
  if (x1 > x2) {print x2} else {print x1}
}
}' > pgain.$$

paste $fic p3.$$ pgain.$$ > pgain2.$$

echo "filter significant associations"
cat pgain2.$$ | awk '{
if($17 > (10/5E-8) ){print}
}' > $fic.pgain

# cleanup
rm *.$$

echo "number of records is "`wc -l $fic.pgain`
EOF

chmod +x compute_pgain.sh

for f in nratio.*.nratio.glm.linear ; do

  ./compute_pgain.sh $f

done

head -1 nratio.1.nratio.glm.linear.pgain >  nratio.glm.linear.pgain
cat nratio.*.nratio.glm.linear.pgain | grep -v 'CHROM' | sort -g -k 17 -r >> nratio.glm.linear.pgain

compute p-gain for nratio.1.nratio.glm.linear
checking if files are present
checking file sizes
getting pvalues
computing pgain
filter significant associations
number of records is 2 nratio.1.nratio.glm.linear.pgain
compute p-gain for nratio.10.nratio.glm.linear
checking if files are present
checking file sizes
getting pvalues
computing pgain
filter significant associations
number of records is 12 nratio.10.nratio.glm.linear.pgain
compute p-gain for nratio.11.nratio.glm.linear
checking if files are present
checking file sizes
getting pvalues
computing pgain
filter significant associations
number of records is 86 nratio.11.nratio.glm.linear.pgain
compute p-gain for nratio.12.nratio.glm.linear
checking if files are present
checking file sizes
getting pvalues
computing pgain
filter significant associations
number of records is 1 nratio.12.nratio.glm.linear.pgain
compute p-gain for nratio.13.nratio.glm.linear
checking if files are present
checking file sizes
getting pvalues
computing pgain

In [60]:
# upload results
# tar cvfz nratio.tgz nratio.*.glm.linear 
# dx upload nratio.tgz
dx upload nratio.glm.linear.pgain

[===========================================================>] Uploaded 22,167 of 22,167 bytes (100%) nratio.glm.linear.pgain
ID                    file-GFyYK00J13J6FX564vgx7Z45
Class                 file
Project               project-G6v874QJ13J49Z2B0gz538G2
Folder                /
Name                  nratio.glm.linear.pgain
State                 closing
Visibility            visible
Types                 -
Properties            -
Tags                  -
Outgoing links        -
Created               Wed Aug 17 18:44:16 2022
Created by            ksuhre
 via the job          job-GFyQ6B8J13J5QJX94jxP6FJX
Last modified         Wed Aug 17 18:44:18 2022
Media type            
archivalState         "live"
cloudAccount          "cloudaccount-dnanexus"
